In [1]:
import pandas as pd
from tqdm.auto import tqdm
import numpy as np
from datetime import date, timedelta
import time
from google.colab import drive
drive.mount('/content/drive')
import random
import matplotlib.pyplot as plt
from scipy.stats.mstats import winsorize
from sklearn.cluster import KMeans

Mounted at /content/drive


In [2]:
!pip install pycaret
import seaborn as sns
import warnings
warnings.filterwarnings(action='ignore')
from pycaret.classification import *

     |████████████████████████████████| 256kB 3.9MB/s 
     |████████████████████████████████| 1.8MB 39.9MB/s 
     |████████████████████████████████| 102kB 9.3MB/s 
     |████████████████████████████████| 157.5MB 36kB/s 
     |█████████████████████████▎      | 52.0MB 1.2MB/s eta 0:00:12ERROR: Exception:
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/pip/_vendor/urllib3/response.py", line 425, in _error_catcher
    yield
  File "/usr/local/lib/python3.6/dist-packages/pip/_vendor/urllib3/response.py", line 507, in read
    data = self._fp.read(amt) if not fp_closed else b""
  File "/usr/local/lib/python3.6/dist-packages/pip/_vendor/cachecontrol/filewrapper.py", line 62, in read
    data = self.__fp.read(amt)
  File "/usr/lib/python3.6/http/client.py", line 463, in read
    n = self.readinto(b)
  File "/usr/lib/python3.6/http/client.py", line 507, in readinto
    n = self.fp.readinto(b)
  File "/usr/lib/python3.6/socket.py", line 586, in readinto
    re

ModuleNotFoundError: ignored

In [ ]:
train_err = pd.read_csv('/content/drive/Shareddrives/dacon/404/train_err2.csv')
train_qui = pd.read_csv('/content/drive/Shareddrives/dacon/404/train_qui2.csv')
train_prob = pd.read_csv('/content/drive/Shareddrives/dacon/404/train_prob2.csv')

test_err = pd.read_csv('/content/drive/Shareddrives/dacon/404/test_err2.csv')
test_qui = pd.read_csv('/content/drive/Shareddrives/dacon/404/test_qui2.csv')

In [ ]:
path = '/content/drive/Shareddrives/dacon/404/preprocessed/mk2_with_categorized/'

test_err_42_counts = pd.read_csv(path + '테스트_에러_42항목별_카운트.csv', index_col=['Unnamed: 0'])
train_err_42_counts = pd.read_csv(path + '트레인_에러_42항목별_카운트.csv', index_col=['Unnamed: 0'])

# 얘는 넣어봐야 의미가 없어서... 일단 제외
test_err_code_frq = pd.read_csv(path + '테스트_에러_에러코드별_발생횟수.csv', index_col=['Unnamed: 0'])
train_err_code_frq = pd.read_csv(path +'트레인_에러_에러코드별_발생횟수.csv', index_col=['Unnamed: 0'])

test_err_sum_byday = pd.read_csv(path +'테스트_에러발생_일일누적합계.csv', index_col=['Unnamed: 0'])
train_err_sum_byday = pd.read_csv(path +'트레인_에러발생_일일누적합계.csv', index_col=['Unnamed: 0'])

# 얘도 넣어봐야 의미가 없어서... 일단 제외
test_qui_frq = pd.read_csv(path +'테스트_퀼리티_항목별_많이등장.csv', index_col=['Unnamed: 0'])
train_qui_frq = pd.read_csv(path +'트레인_퀼리티_항목별_많이등장.csv', index_col=['Unnamed: 0'])

test_qui_sum_byday = pd.read_csv(path +'테스트_퀼리티로그_일일수집합계.csv', index_col=['Unnamed: 0'])
train_qui_sum_byday = pd.read_csv(path +'트레인_퀼리티로그_일일수집합계.csv', index_col=['Unnamed: 0'])

test_qui_sum_byquality = pd.read_csv(path +'테스트_퀼리티_각각카테고리_합산.csv', index_col=['index'])
train_qui_sum_byquality = pd.read_csv(path +'트레인_퀼리티_각각카테고리_합산.csv', index_col=['index'])

test_2h = pd.read_csv(path +'테스트_err_2시간간격.csv', index_col=['Unnamed: 0'])
train_2h = pd.read_csv(path +'트레인_err_2시간간격.csv', index_col=['Unnamed: 0'])

train_6h = pd.read_csv(path + '트레인_err_6시간간격.csv', index_col=['Unnamed: 0'])
test_6h = pd.read_csv(path + '테스트_err_6시간간격.csv', index_col=['Unnamed: 0'])

train_12h = pd.read_csv(path + '트레인_err_12시간간격.csv', index_col=['Unnamed: 0'])
test_12h = pd.read_csv(path + '테스트_err_12시간간격.csv', index_col=['Unnamed: 0'])

In [ ]:
train_time_series_6h = pd.read_csv(path + '트레인_err_시계열.csv', index_col=['Unnamed: 0'])
test_time_series_6h = pd.read_csv(path + '테스트_err_시계열.csv', index_col=['Unnamed: 0'])

In [ ]:
train_time_series_6h

In [ ]:
train_2h.rename(columns = lambda x: x + '_2', inplace = True)
test_2h.rename(columns = lambda x: x + '_2', inplace = True)

train_6h.rename(columns = lambda x: x + '_6', inplace = True)
test_6h.rename(columns = lambda x: x + '_6', inplace = True)

train_12h.rename(columns = lambda x: x + '_12', inplace = True)
test_12h.rename(columns = lambda x: x + '_12', inplace = True)

In [ ]:
# 각 사용자가 42가지 에러타입을 몇 번씩 겪었는지 나타냄
train_err_42_counts

In [ ]:
# 각 사용자가 퀼리티 1 부터 퀼리티 12까지 자주 겪은 퀼리티 로그를 나타냄
# _1은 가장 자주 겪은 로그이며, _2는 두 번째로 자주 겪은 로그임. (None은 2번째 로그가 없음, 한 가지 오류만 겪음)
train_qui_frq

################ 이거는 카테고리 기준 등장빈도이며... 이를 확률로 치환하는 새로운 방법을 시도해본다.

In [ ]:
# 일단 자료들 날짜형식이나 콤마(,) 제거 등 간단한 전처리 진행

train_qui2 = train_qui.copy()
test_qui2 = test_qui.copy()
train_err2 = train_err.copy()
test_err2 = test_err.copy()

# 퀼리티 3,4는 값이 1밖에 없어서 의미가 없기 때문에 제외
# 펌웨어 버전은 이미 err 자료에 있기 때문에 중복으로 제외
# 퀼리티 0은 EDA 결과 퀼리티 2와 피어슨 상관계수 0.99를 보이기 때문에 0과 2 중에 하나만 골랐음
# 0의 결측치 Nan은 15%고 2의 결측치는 4%기 때문에 결측치가 적은 2를 남기고 0은 drop 시킨다.

## 퀼리티 6과 7도 0.81로 강한 상관관계를 보이고 있는데, 하나를 제외해야 하나 고민이 됨
train_qui2.drop(['fwver','quality_0', 'quality_3', 'quality_4'], axis =1, inplace=True)
test_qui2.drop(['fwver', 'quality_0', 'quality_3', 'quality_4'], axis =1, inplace=True)

test_qui2 = test_qui2.replace(',','', regex=True)
train_qui2 = train_qui2.replace(',','', regex=True)
test_qui2.loc[:,'quality_1':'quality_12'] = test_qui2.loc[:,'quality_1':'quality_12'].astype(float)
train_qui2.loc[:,'quality_1':'quality_12'] = train_qui2.loc[:,'quality_1':'quality_12'].astype(float)

train_qui2['time'] = pd.to_datetime(train_qui['time'], format = '%Y-%m-%d %H:%M:%S')
test_qui2['time'] = pd.to_datetime(test_qui['time'], format = '%Y-%m-%d %H:%M:%S')

train_err2['time'] = pd.to_datetime(train_err2['time'], format = '%Y-%m-%d %H:%M:%S')
test_err2['time'] = pd.to_datetime(test_err2['time'], format = '%Y-%m-%d %H:%M:%S')

train_err4 = train_err2.copy()
test_err4 = test_err2.copy()

In [ ]:
# 퀼리티 임계치
# [-1, 0, 1] == -1 카테고리 -1, -1~0 =카테고리 0, 그 이상 = 카테고리 1
qualities = {}

qualities['quality_1'] = [-1,0, 1]
qualities['quality_2'] = [-1,0, 1]
qualities['quality_5'] = [-1,0,1,2,4, 10]
qualities['quality_6'] = [-1,0, 1]
qualities['quality_7'] = [0,1,100, 101]
qualities['quality_8'] = [0,1,2]
qualities['quality_9'] = [0,1,2]
qualities['quality_10'] = [0,1,2,3, 4, 5,7, 10 ,14,15] ##??
qualities['quality_11'] = [-1,0,1]
qualities['quality_12'] = [0,1,2]

## 모델 임계치
## 모델 0,1,2,3,4의 비율을 더하면 99.999% 이므로, 모델 5~8은 other model로 통합하는 것이 바람직 해 보임.

In [ ]:
def quality_categorize(dataset):
  data = dataset.copy()
  for key in tqdm(qualities):
    intervals = qualities.get(key)
    for i in range(len(intervals)):
      if intervals[i] == intervals[-1]:
        data[key][data[key] >= intervals[i]] =  intervals[i]
      # elif intervals[i+1] - intervals[i] == 1:
      #   continue
      else:
        data[key][(data[key] >= intervals[i]) & (data[key] < intervals[i+1])] = intervals[i]
  return data

###### 그냥 쓰자니 불일치가 너무 많아서... 카테고리화 해서 확률로 전환한다.

train_qui3 = quality_categorize(train_qui2)
test_qui3 = quality_categorize(test_qui2)

In [ ]:
prob_list = list(set(train_prob.user_id))

train_qui3['problems'] = train_qui3.user_id.isin(prob_list)
train_qui3_tr = train_qui3[train_qui3['problems']].dropna()
train_qui3_fls = train_qui3[train_qui3['problems'] != True].dropna()

In [ ]:
def probability(trainset, testset):
  train = trainset.copy()
  test = testset.copy()

  cols = list(train.loc[:,'quality_1':'quality_12'].columns)

  train['problems'] = train.user_id.isin(prob_list)
  train_tr = train[train['problems']]
  train_fls = train[train['problems'] != True]


  for col in tqdm(cols):
    temp_ori = train[col].value_counts()
    temp_tr = train_tr[col].value_counts()
    temp_fls = train_fls[col].value_counts()

    ori = pd.DataFrame(temp_ori)
    tr = pd.DataFrame(temp_tr)

    probs = tr/ori
    for ind in list(probs.index):
      train[col][train[col] == ind] = probs.loc[ind,:].values[0]
      test[col][test[col] == ind] = probs.loc[ind,:].values[0]
  train.drop(['problems'], axis = 1, inplace = True)
  return train, test, probs

In [ ]:
train_qui_prob, test_qui_prob, prob = probability(train_qui3, test_qui3)

In [ ]:
# 일단 확률로 바꾼 값들을 같은 ID끼리 평균으로 묶는다.

train_qui_probs = train_qui_prob.groupby(by = 'user_id').mean()
test_qui_probs = test_qui_prob.groupby(by = 'user_id').mean()

In [ ]:
#################### test_qui_frq 대신에 이걸 넣자. ########################
########## 넣어봤는데 그닥... 이번에는 단순 합계를 넣어보자  test_qui_sum_byquality


test_qui_probs

In [ ]:
### 펌웨어의 확률을 구하는 함수 ###
## 유진's 코드를 참조하여 좀더 빠르게 구동되도록 수정함.

def fw_probs(traindata, testdata):
  
  train_temp = traindata.drop_duplicates(subset = 'user_id')
  test_temp = testdata.drop_duplicates(subset = 'user_id')
  

  test_list = []
  train_temp['problems'] = train_temp['user_id'].isin(prob_list)
  train_tr = train_temp[train_temp['problems']]
  train_fls = train_temp[train_temp['problems'] != True]

  new_fw = ['03.11', '04.16', '04.22', '04.33', '04.73', '04.82', '05.15', '05.66', '8.5', '10']

  for fw in tqdm(new_fw):
    if fw == '10':
      ori = len(train_temp[(train_temp['fwver'].str.contains(fw)) & ~(train_temp['fwver'].str.contains('104')) & ~(train_temp['fwver'].str.contains('109'))])
      tr = len(train_tr[(train_tr['fwver'].str.contains(fw)) & ~(train_tr['fwver'].str.contains('104')) & ~(train_tr['fwver'].str.contains('109'))])
      fls = len(train_fls[(train_temp['fwver'].str.contains(fw)) & ~(train_fls['fwver'].str.contains('104')) & ~(train_fls['fwver'].str.contains('109'))])
      fw_st = [ori, tr, fls, tr/ori]
      test_list.append(fw_st)
    else:
      ori = len(train_temp[train_temp['fwver'].str.contains(fw)])
      tr = len(train_tr[train_tr['fwver'].str.contains(fw)])
      fls = len(train_fls[train_fls['fwver'].str.contains(fw)])
      fw_st = [ori, tr, fls, tr/ori]
      test_list.append(fw_st)

  fw_df = pd.DataFrame(test_list, index = new_fw, columns = ['원본', '불만고객', '고객', '불만확률'])

  train_temp['fw_prob'] = 0
  test_temp['fw_prob'] = 0

  for fw in tqdm(new_fw):
    if fw == '10':
      train_temp['fw_prob'][(train_temp['fwver'].str.contains(fw)) & ~(train_temp['fwver'].str.contains('104')) & ~(train_temp['fwver'].str.contains('109'))] = fw_df.loc[fw][3]
      test_temp['fw_prob'][(test_temp['fwver'].str.contains(fw)) & ~(test_temp['fwver'].str.contains('104')) & ~(test_temp['fwver'].str.contains('109'))] = fw_df.loc[fw][3]
    else:
      train_temp['fw_prob'][train_temp['fwver'].str.contains(fw)] = fw_df.loc[fw][3]
      test_temp['fw_prob'][test_temp['fwver'].str.contains(fw)] = fw_df.loc[fw][3]
  df = train_temp.loc[:,['user_id', 'fw_prob']]
  df_t = test_temp.loc[:,['user_id', 'fw_prob']]

  df.set_index('user_id', inplace = True)
  df_t.set_index('user_id', inplace = True)

  return df, df_t

In [ ]:
train_fw, test_fw = fw_probs(train_err2, test_err2)

In [ ]:
# Error 데이터를 일일 단위로 쪼개서 각 사용자가 날짜별로 겪은 에러코드의 누계
# 0일은 10월 31일, 마지막은 11월 30일
train_err_sum_byday

In [ ]:
# Quality 데이터를 일일 단위로 쪼개서 퀼리티 로그가 사용자 별로 몇 번 수집되었나
# 누적, 마지막 11월 30일 자료는 최종 몇 번의 로그 수집이 있었는가를 의미함.
train_qui_sum_byday

In [ ]:
# train_err_42_counts, train_err_code_frq, train_qui_frq, train_err_sum_byday, train_qui_sum_byday
# test_err3, test_err_42_counts, test_err_code_frq, test_qui_frq, test_err_sum_byday, test_qui_sum_byday
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
robust = RobustScaler()
robust_q = RobustScaler()
robust_nq = RobustScaler()
minmax = MinMaxScaler()

In [ ]:
test_qui_frq = test_qui_frq.astype(str)
train_qui_frq = train_qui_frq.astype(str)

In [ ]:
# 24시간을 단순하게 평균내면, 23시와 1시의 평균이 이상해지기 때문에(실제로는 2시간 차이인데, 산술적으로 22시간 차이로 잡힘)
# 태양광 때 처럼 일단 코사인 시간으로 바꿔서 평균, 분산, 최빈값을 구한다.

def cos_time(dataset):
  dataset['cos_time'] = np.cos(2*np.pi*dataset.hour/24)
  dataset['sin_time'] = np.sin(2*np.pi*dataset.hour/24)
  return dataset

def cos_stats(dataset):
  dataset['hour'] = dataset['time'].dt.hour
  data = cos_time(dataset)[['user_id', 'cos_time', 'sin_time']]
  cols = []
  temp = data.groupby(by = 'user_id')

  a = temp['cos_time'].mean()
  cols += ['cos_mean']
  b = temp['cos_time'].std()
  cols += ['cos_std']

  c = temp['sin_time'].mean()
  cols += ['sin_mean']
  d = temp['sin_time'].std()
  cols += ['sin_std']

  df = pd.concat([a,b,c,d], axis = 1) 
  df.columns = cols
  return df

In [ ]:
cos_train = cos_stats(train_err4)
cos_test = cos_stats(test_err4)

In [ ]:
cos_test

In [ ]:
# errtype 1~42까지 확률로 치환하기.

def errtype_to_prob(trainset, testset):
  data = trainset.copy()
  t_data = testset.copy()
  data['problems'] = data['user_id'].isin(prob_list)
  one_hot = pd.get_dummies(data['errtype'])

  temp = data['problems']
  temp2 = pd.concat([temp, one_hot], axis = 1)

  temp2_tr = temp2[temp2['problems']].fillna(0)
  temp2_fls = temp2[temp2['problems'] != True].fillna(0)

  trues = pd.DataFrame(temp2_tr.mean()).transpose()
  falses = pd.DataFrame(temp2_fls.mean()).transpose()
  tru = trues.drop('problems', axis = 1)
  fls = falses.drop('problems', axis = 1)
  rslt = tru/(tru+fls)
  er_list = list(set(data.errtype))
  er_list.sort()
  for er in tqdm(er_list):
    data.errtype[data['errtype'] == er] =rslt.loc[0,er]
    t_data.errtype[t_data['errtype'] == er] =rslt.loc[0,er]
  return data, t_data

In [ ]:
train, test = errtype_to_prob(train_err2, test_err2)

In [ ]:
train_errtype_prob = train.groupby(by = 'user_id').mean()[['errtype']]
test_errtype_prob = test.groupby(by = 'user_id').mean()[['errtype']]

In [ ]:
# 중간에 펌웨어 업데이트를 한 경우에 1, 아니면 0으로 표시하는 함수 

def update(dataset):

  data = dataset.copy()
  ids = list(set(data.user_id))
  ids.sort()
  print('1/3')
  data.replace({'fwver': np.nan}, {'fwver': '00.00.0000'}, inplace = True)
  data.replace({'fwver': '8.5.3'}, {'fwver': '08.05.3000'}, inplace = True)
  data.replace({'fwver': '10'}, {'fwver': '10.00.0000'}, inplace = True)
  data['fwver'] = data['fwver'].str.replace('\.','', regex = True)
  print('2/3')
  data = data.astype({'fwver': 'int'})
  fw_list = data.fwver.unique()
  data2 = data.groupby(by = 'user_id').mean()
  print('3/3')
  data2['fw_update'] = data2['fwver'].isin(fw_list)
  data2['fw_update'] = data2['fw_update'].astype(int)
  df = data2.loc[:,['fw_update']]
  return df

In [ ]:
fw_update_train = update(train_err2)
fw_update_test = update(test_err2)

In [ ]:
train_err3 = train_err2[['user_id', 'errtype', 'errcode', 'time']]
test_err3 = test_err2[['user_id', 'errtype', 'errcode', 'time']]

# 상위 15개 리스트
err_code =['1','0','connection timeout','B-A8002','80','79','14','active','2','84','85','standby','NFANDROID2','connection fail to establish','3']

# 위의 15개에 포함되지 않는 에러코드는 걍 'etc'로 분류
train_err3['errcode'][~train_err3['errcode'].isin(err_code)] = 'ETC'
test_err3['errcode'][~test_err3['errcode'].isin(err_code)] = 'ETC'

In [ ]:
dummi = pd.get_dummies(train_err3['errcode'], prefix = 'Err_code_')
id = train_err3[['user_id']]
train_types = pd.concat([id, dummi], axis =1)

dummi = pd.get_dummies(test_err3['errcode'], prefix = 'Err_code_')
id = test_err3[['user_id']]
test_types = pd.concat([id, dummi], axis =1)

In [ ]:
test_err_types = test_types.groupby(by ='user_id').sum()
train_err_types = train_types.groupby(by ='user_id').sum()

In [ ]:
# 시계열 자료를 활용해 군집화.
ks = KMeans(n_clusters = 7, random_state = 42)
ks.fit(train_time_series_6h)
train_time_series_6h['Group'] = ks.predict(train_time_series_6h)
# train_time_series_6h['problems'] = train_time_series_6h.index.isin(prob_list)

In [ ]:
train_time_series_6h

In [ ]:
train_time_series_6h.drop('problems', axis = 1, inplace = True)

In [ ]:
trainset = pd.concat([train_fw, train_err_42_counts,train_err_types, train_qui_sum_byday, train_err_sum_byday, train_qui_probs, cos_train, train_errtype_prob, fw_update_train, train_2h], axis = 1) #train_err_sum_byday, train_qui_sum_byquality
testset = pd.concat([test_fw, test_err_42_counts, test_err_types, test_qui_sum_byday, test_err_sum_byday,test_qui_probs, cos_test, test_errtype_prob, fw_update_test, test_2h], axis = 1) #test_err_sum_byday, test_qui_sum_byquality, test_qui_probs


# train_err_time_stat_2h
# test_err_time_stat_2h
# trainset.reset_index(inplace=True)
# testset.reset_index(inplace=True)
# trainset = trainset.dropna(axis = 1)
# testset = testset.dropna(axis = 1)
# trainset = trainset.drop(['model_nm', 'fwver'], axis = 1)
# testset = testset.drop(['model_nm','fwver'], axis = 1)

In [ ]:
# quantile = 0.03

# trainset_w = trainset.copy()
# testset_w = testset.copy()
# train_temp = trainset_w.loc[:,'0':'er_sum_30']
# test_temp = testset_w.loc[:,'0':'er_sum_30']

# trainset_w.loc[:,'0':'er_sum_30'] = pd.DataFrame(winsorize(train_temp, limits = [quantile,quantile], axis = 1), index = train_temp.index, columns = train_temp.columns)
# testset_w.loc[:,'0':'er_sum_30'] = pd.DataFrame(winsorize(test_temp, limits = [quantile,quantile], axis = 1), index = test_temp.index, columns = test_temp.columns)

In [ ]:
# trainset2 = np.log(trainset_w + 2)
# testset2 = np.log(testset_w + 2)

In [ ]:
def nan_to_mean(dataset):
  data = dataset.copy()  
  col_list = list(dataset.columns)

  for col in tqdm(col_list):
    temp = data[col]
    val = temp.mean()
    val_list = []
    for id in range(45000):
      a = random.uniform(0.985,1.015)
      val_list.append(val * a)
    temp.fillna(value = pd.Series(val_list), inplace=True)
    data[col] = temp
  return data

In [ ]:
# trainset2 = nan_to_mean(trainset)
# testset2 = nan_to_mean(testset)

# trainset2 = trainset.fillna(0)
# testset2 = testset.fillna(0)

# trainset2 = trainset.copy()
# testset2 = testset.copy()

In [ ]:
 trainset2 = np.log(trainset + 2)
 testset2 = np.log(testset + 2)

# trainset2 = pd.DataFrame(robust.fit_transform(trainset2), columns=trainset2.columns, index = trainset2.index)

# testset2 = pd.DataFrame(robust.transform(testset2), columns=testset2.columns, index = testset2.index)

# trainset2 = pd.DataFrame(minmax.fit_transform(trainset2), columns=trainset.columns, index = trainset.index)
# testset2 = pd.DataFrame(minmax.fit_transform(testset2), columns=testset.columns, index = testset.index)
# trainset2 = trainset.copy()
# testset2 = testset.copy()

trainset3 = pd.concat([trainset2, train_time_series_6h], axis = 1)
testset3 = pd.concat([testset2, train_time_series_6h], axis = 1)

In [ ]:
# trainset2 = trainset2.reset_index()
# probs = list(set(train_prob.user_id))
# trainset2['TARGET'] = trainset2['index'].apply(lambda x: 1 if x in probs else 0)
# trainset2.set_index('index', inplace=True)
# trinset3 =trainset2

trainset3.reset_index(inplace=True)
probs = list(set(train_prob.user_id))
trainset3['TARGET'] = trainset3['index'].apply(lambda x: 1 if x in probs else 0)
trainset3.set_index('index', inplace = True)

num = list(trainset3.columns)
num.remove('TARGET')
num.remove('Group')

In [ ]:
trainset3

In [ ]:
len(num)

In [66]:
clf = setup(data = trainset3, target = 'TARGET', session_id = 42)#, numeric_features=num)#, categorical_features=['Group']) 
cat = create_model('catboost')

IntProgress(value=0, description='Processing: ', max=3)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,14:07:38
Status,. . . . . . . . . . . . . . . . . .,Preprocessing Data


Text(value="Following data types have been inferred automatically, if they are correct press enter to continue…

,Data Type
fw_prob,Numeric
0,Numeric
1,Numeric
2,Numeric
3,Numeric
...,...
2020-12-02 06:00:00,Numeric
2020-12-02 12:00:00,Numeric
2020-12-02 18:00:00,Categorical
Group,Categorical


AttributeError: ignored

In [ ]:
from sklearn.preprocessing import power_transform

In [ ]:
(trainset3.min()).min()

-1291939733407315.2

In [ ]:
data.max()

336.481332480508

In [ ]:
train_qui

,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12,time
0,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0,2020-11-29 09:00:00
1,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0,2020-11-29 09:00:00
2,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0,2020-11-29 09:00:00
3,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0,2020-11-29 09:00:00
4,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0,2020-11-29 09:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
828619,24997,04.22.1778,0.0,0,0.0,0,0,1,0,0,0,0,17,0,0,2020-11-24 03:30:00
828620,24997,04.22.1778,0.0,0,0.0,0,0,0,0,0,0,0,17,0,0,2020-11-24 03:30:00
828621,24997,04.22.1778,0.0,0,0.0,0,0,3,0,0,0,0,17,0,0,2020-11-24 03:30:00
828622,24997,04.22.1778,0.0,0,0.0,0,0,0,0,0,0,0,17,0,0,2020-11-24 03:30:00


In [ ]:
train_qui2

,quality_5
index,
10000,1.711834
10001,NaN
10002,1.708805
10003,NaN
10004,1.722691
...,...
24995,1.885688
24996,NaN
24997,1.815850


In [ ]:
# tuned_cat = tune_model(cat, optimize='AUC')

# 앙상블로 예측하고 싶으면 model = final_model, 캣부로 해보고 싶으면 model = tuned_cat
model = cat

# predictions = predict_model(model, data = testset2)

In [ ]:
x = []
for i in range(len(predictions['Score'])):
  if predictions['Label'][i+30000] ==1:
    x.append(predictions['Score'][i+30000])
  else:
    x.append(1-predictions['Score'][i+30000])

In [ ]:
df = pd.DataFrame(model.feature_importances_, index = model.feature_names_, columns = ['feature'])

In [ ]:
df = df.reset_index()

In [ ]:
df.to_csv("F_impo2.csv", index = False)
!cp F_impo2.csv "drive/My Drive/"

In [ ]:
sample_submssion = pd.read_csv('/content/drive/Shareddrives/dacon/404/sample_submission.csv')
sample_submssion['problem'] = x
sample_submssion.to_csv("CAT.csv", index = False)
!cp CAT.csv "drive/My Drive/"

In [ ]:
sample_submssion

,user_id,problem
0,30000,0.957976
1,30001,0.325178
2,30002,0.509332
3,30003,0.908938
4,30004,0.768151
...,...,...
14994,44994,0.180381
14995,44995,0.398818
14996,44996,0.695316
14997,44997,0.943637
